## Tutorial 3 - Convex Optimization

**Objectives:**

* To implement the constrained linear logistic regression.


**Instructions:**
   
* If you are using Jupyter intalled on your computer, Go to File->Open. Drag and drop "lab3.ipynb" file to the home interface and click upload. 
* If you are using Google Colaboratory, Click File->Upload notebook, and and upload "lab3.ipynb" file
* Complete exercises in "Lab03.ipynb".
* To run the cell you can press Ctrl-Enter or hit the Play button at the top.
* Complete all exercises marked with **TODO**.
* Save your file when you are done with the exercises, so you can show your tutor next week.


**Authors**
* Kloe and Lydia

## 1. Linear Logistic Regression
* Let $\{(\mathbf{x}_1,y_1),\cdots,(\mathbf{x}_n,y_n)\}$ be $n$ training examples. Denote $\mathbf{x}_i\in \mathbb{R}^d$ as $i$-th feature and $y_i$ as its label. Our target is to learn a function $f(\mathbf{x})$ from the training examples such that $f$ can predict the label $y$ of any feature $\mathbf{x}$.

* In linear logistic regression, we let $f(x)=\mathbf{w}^{\top}\mathbf{x}$ where $\mathbf{w}$ is the parameter of the function. 

* In order to learn a good $f$, according to tutorial 2, we usually solve the following problem:

* $\begin{equation} \min_{\mathbf{w}\in\mathbb{R}^d} NLL(\mathbf{w})=\sum_{i=1}^n \log(1+\exp(-y_i f(x_i,\mathbf{w}))). \end{equation}$

### 1.1 Constraints on the Parameters
* When we optimize above linear logistic regression problem, sometimes, some entries of the parameter $\mathbf{w}$ may become very large such that this model can memorize the training set very well. But the classifier may not generalize well on the test set. In this condition, we need to add some constraints on the parameters. 
* For example, we can assume that the parameters reside on a small sphere with radius $\sqrt{\gamma}$, i.e,
* $\begin{equation} \begin{split} \min_{\mathbf{w}\in\mathbb{R}^d} \quad &NLL(\mathbf{w})=\sum_{i=1}^n \log(1+\exp(-y_i f(x_i,\mathbf{w}))), \\ \textrm{subject to} \quad & \sum_{i=1}^d w_i^2 = \gamma. \end{split} \end{equation}$
* This is a constrained convex optimization problem.

### 1.2 Lagrangian Function and ADMM
* The Lagrangian function is:
* $\begin{equation}  \min_{\mathbf{w}\in\mathbb{R}^d} \max_{\lambda} \quad \mathcal{L}(\mathbf{w},\lambda)=\sum_{i=1}^n \log(1+\exp(-y_i f(x_i,\mathbf{w}))) + \lambda(\sum_{i=1}^d w_i^2 - \gamma). \end{equation}$
* $\lambda$ is known as the Lagrangian multiplier. 
* Here, we solve the primal-dual problem using alternating direction method of multipliers (ADMM). Specifically, in each iteration, we update $\mathbf{w}$ using the gradient descent method and update $\lambda$ using the gradient ascent method, which is shown as following:
* $\begin{equation} \begin{split} \mathbf{w}^{(k+1)} &= \mathbf{w}^{(k)}-\alpha_1 \frac{\partial \mathcal{L}}{\partial \mathbf{w}}|_{\mathbf{w}^{(k)}}; \\  \lambda^{(k+1)} &= \lambda^{(k)} + \alpha_2 \frac{\partial \mathcal{L}}{\partial \lambda}|_{\lambda^{(k)}}. \end{split} \end{equation}$
* Here, $k$ refers to the number of iterations, $\alpha_1$ and $\alpha_2$ are step sizes for updating $\mathbf{w}$ and $\lambda$, respectively.

### 1.3 Implementation of ADMM Algorithm
* Now we start to implement our method.
* Here, we also take the binary classification as an example. We use the Iris example in tutorials 1 and 2.

#### 1.3.1 Load Iris Dataset

In [16]:
using CSV
data = CSV.read("iris.csv")

features = Array{Float64}[]
target = []
for i in 1:size(data,1)
    if data.species[i] == "setosa"
        push!(features,[data.petal_length[i],data.petal_width[i],data.sepal_length[i],data.sepal_width[i]])
        push!(target,1)
    elseif data.species[i] == "versicolor"
        push!(features,[data.petal_length[i],data.petal_width[i],data.sepal_length[i],data.sepal_width[i]])
        push!(target,-1)
    end
end

n = length(target)
dim = 4
X = zeros(n,dim)
Y = zeros(n)
for i in 1:n
    for j in 1:dim
        X[i,j] = features[i][j]
    end
    Y[i] = target[i]
end

#### 1.3.2 Split the Dataset
* Note that, to avoid overfitting or to conduct model selection, we usually split the dataset to training, validation, and test data. 
* Here, for simplicity, we directly split the data to training and test data. Must remember that for academic research, validation set is very important and necessary.
* More details about training, test and validation sets can be found at https://en.wikipedia.org/wiki/Training,_test,_and_validation_sets.

In [57]:
using Random 
# shuffle index
rng = Random.seed!()
arr = shuffle(rng,1:n)

# split the data to training (80%) and test data (20%)
n_train = floor(Int,0.8*n)
n_test = n - n_train

# training data
x_train = X[arr[1:n_train],:]
x_train = vcat(x_train',ones(1,n_train)) # add offset to data points because we consider the bias in the model
y_train = Y[arr[1:n_train],:]

# test data
x_test = X[arr[n_train+1:end],:]
x_test = vcat(x_test',ones(1,n_test))
y_test = y[arr[n_train+1:end]]
print(size(x_test))

(5, 20)

#### 1.3.3 Compute the Gradient of Lagrangian Function

**Exercise 1.3.1**
* Finish the code for computing graident of $\mathcal{L}$ with respect to $\mathbf{w}$.

In [40]:
function grad_lambda(w,gamma)
    """Compute the gradient of Lagrangian function with respect to lambda.
    Input:
        w: array, shape = [d+1]. The parameter of the classifier.
        gamma: real value. The square of the radius of the sphere.
    Output:
        grad: real value. The gradient of L with respect to lambda.
    """
    return norm(w)^2 - gamma 
end

function grad_w(X,Y,w,labda)
    """Compute the gradient of Lagrangian function with respect to w.
    Input:
        X: array, shape = [d+1,n], features
        Y: array, shape = [n], labels
        w: array, shape = [d+1], current value of w.
        labda: real value, the dual variable lambda.
    Output:
        grad: array, shape = [d+1]
            The gradient of L with respect to w.
    """
    d,n = size(X)
    grad = zeros(d)
    
    temp = zeros(n)
    for i in 1:n
        u = 1.0/(1.0+exp(-dot(X[:,i],w)))
        if Y[i] == 1
            temp[i] = u-1
        else
            temp[i] = u
        end
    end
    grad = X*temp + 2*labda*w
    
    return grad
end

grad_w (generic function with 1 method)

#### 1.3.4 Optimization of the Overall Problem

** Exercise 1.3.2**
* Finish the overall algorithm

In [41]:
function constrainedLR_admm(X,Y,gamma=0.1,alpha_1 = 1e-3,alpha_2=1e-0, maxiter=500, tol=1e-8)
    """Optimizing the constrained logistic regression using ADMM
    Input: 
       X: array, shape = [d+1, n], Features.
       Y: array, shape = [n], Labels.
       gamma: real value. The square of the radius of the sphere.
       alpha_1 and alpha_2: real values. The step size or learning rate for both w and lambda.
       maxiter: real value. The maximum number of iterations.
       tol: real value. Stop criteria.
    Output:
       w: array, shape = [d+1]. The estimated w.
       lamb: real value. The estimated lambda.
    """
    # Initialise w and lambda 
    d,n = size(X)
    w = 0.01*rand(d)
    labda = 0
    
    # we use the norm of gradient of L w.r.t. w as a stop criteria.
    # If it is smaller than tol, then update stops.
    diff_ = Inf
    iters = 0
    
    while (iters < maxiter) & (diff_ > tol)
        # first: update w
        gradw = grad_w(X,Y,w,labda)
        w -= alpha_1 * gradw
        diff_ = norm(gradw)
        
        # second: update labda
        gradlambda = grad_lambda(w,gamma)
        labda += alpha_2 * gradlambda

        iters += 1
    end
    
    return w, labda
end

w,labda = constrainedLR_admm(x_train,y_train)
println(w)
println(labda)
print(norm(w)^2)
    

[-0.277959, -0.115293, 0.0346117, 0.171608, 0.0279012]
59.82324645682874
0.12197965660900956

#### 1.3.5 Prediction and Test Accuracy
* Test the performance of our method.

In [60]:
function predict(w,X,Y)
    d,n = size(X)
    acc = 0
    for i in 1:n
        u = 1.0/(1.0+exp(-dot(X[:,i],w)))
        if (u>0.5) & (Y[i] == 1)
            acc+=1
        elseif (u<=0.5) & (Y[i] == -1)
            acc+=1
        end
    end
    
    acc /= size(Y)[1]
    
    return acc
end

acc = predict(w,x_test,y_test)
print(acc)

1.0